## Let $\Omega = (1,0)^2$ $\Gamma_D = (0,1) \times \{0\}$ $\Gamma_N$ = \{0\} \time (0,1) and $\Gamma_R = \partial \Omega (\Gamma_N \Gamma_R )$

## $-\Delta u  = f$ 

## $u  = u_D$ on $\Gamma_D$

## $\frac{\partial u}{\partial n} = n$ on $\Gamma_N$

## $\alpha u + \frac{\partial u}{\partial n} = g$ on $\Gamma_R$

## with $f = 1$ $u_D = x$ $\alpha = 5$ $g = y$

## $\Rightarrow -\Delta u = f \Rightarrow -\int_{\Omega} \Delta u v = \int_{\Omega} f v$

## $\Rightarrow -\Delta u v = \nabla u \nabla v - div(\nabla u v)$

## $\int_{\Omega}\nabla u \nabla v - \int_{\Gamma}\frac{\partial u}{\partial n}v = \int_{\Omega}fv$

## $\int_{\Omega}\nabla u \nabla v -5\int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N  \Gamma_R}yv +\int_{\Gamma_D}\frac{\partial x}{\partial n}v$

## we are allowed to set the testfunction v on $\Gamma_D$ to 0

## $\int_{\Omega}\nabla u \nabla v -5\int_{\Gamma_R}uv = \int_{\Omega}v+\int_{\Gamma_N  \Gamma_R}yv$

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [2]:
mesh.GetBoundaries()

('bottom', 'right', 'top', 'left')

In [5]:
source = 1
ud = CoefficientFunction(x)
g = CoefficientFunction([y if bc!="left" else 0 for bc in mesh.GetBoundaries()])
R = CoefficientFunction([5 if bc=="right" or "bottom" else 0 for bc in mesh.GetBoundaries()])
alpha = 5

In [7]:
fesp = H1(mesh, order=4, dirichlet="left")
up, vp = fesp.TnT()

ap = BilinearForm(fesp)
ap += grad(up)*grad(vp)*dx - R*up*vp * ds 
ap.Assemble()

fp = LinearForm(fesp)
fp += source*vp*dx + g*vp * ds
fp.Assemble()

gfup = GridFunction(fesp, "u-primal")
gfup.Set(ud, BND)

r = fp.vec.CreateVector()
r.data = fp.vec - ap.mat * gfup.vec
gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
Draw (gfup)
Draw (R * grad(gfup), mesh, "flux-primal")

In [22]:
order_flux=1
V = HDiv(mesh, order=order_flux, dirichlet="left")
Q = L2(mesh, order=order_flux-1)
fesm = FESpace([V,Q])

In [23]:
sigma, u = fesm.TrialFunction()
tau, v = fesm.TestFunction()
normal = specialcf.normal(mesh.dim)
print (normal)

coef normal vector, real, dim=2



In [24]:
am = BilinearForm(fesm)
am += (sigma*tau - div(sigma)*v + div(tau)*u)*dx
am.Assemble()
fm = LinearForm(fesm)
fm += source*v*dx + ud*(tau.Trace()*normal)*ds + R*u*ds
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")

In [25]:
gfsigma, gfu = gfm.components

In [26]:
gfsigma.Set(CoefficientFunction( (sin(10*x), sin(10*y)) ))
gfu.Set (x)
Draw (gfsigma, mesh, "flux-mixed")
Draw (gfu, mesh, "u-mixed")

In [27]:
gfsigma.Set(g*normal, BND)

In [28]:
# fm.vec.data -= am.mat * gfm.vec
# gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * fm.vec
solvers.BVP(bf=am, lf=fm, gf=gfm)
Redraw()

In [29]:
print ("err-u:   ", sqrt(Integrate( (gfup-gfu)**2, mesh)))
errflux = grad(gfup) - gfsigma
print ("err-flux:", sqrt(Integrate(errflux*errflux, mesh)))

err-u:    0.8667125777890489
err-flux: 1.552367937729188
